### Import modules and init in gee


In [1]:
import pandas as pd
from dateutil import tz

import ee
import geemap

pd.options.mode.copy_on_write = True

Инициализация GEE

In [2]:
geemap.ee_initialize()
oeel = geemap.requireJS()
ee.Authenticate()
ee.Initialize(project='ee-amazyar-test1')
baikal_shape = ee.FeatureCollection('projects/ee-amazyar-test/assets/baikal')

(node:25844) [DEP0040] DeprecationWarning: The `punycode` module is deprecated. Please use a userland alternative instead.
(Use `node --trace-deprecation ...` to show where the warning was created)


Приведение к читаемому виду

In [3]:
def add_timezone(df: pd.DataFrame) -> pd.DataFrame:
   # format = "%d.%m.%Y  %H:%M:%S"
   format = "%Y-%m-%d  %H:%M:%S"
   return df["Time"].apply(lambda dt: pd.to_datetime(dt, format=format).replace(tzinfo=tz.gettz("Asia/Irkutsk")))

In [4]:
df_lst = pd.read_csv('Наземеные_измерения/ready/Temp21-24_cut1.csv')
df_lst["Time"] = add_timezone(df_lst)
df_lst["T"] = df_lst['T'].astype(float)
df_lst["Lat"] = df_lst['Lat'].astype(float)
df_lst["Lon"] = df_lst['Lon'].astype(float)
df_lst

,Time,T,Lat,Lon
0,2021-05-25 18:42:01+08:00,2.56830,51.896156,105.098007
1,2021-05-25 18:43:06+08:00,2.64110,51.895668,105.102013
2,2021-05-25 18:44:11+08:00,2.63280,51.895203,105.106033
3,2021-05-25 18:45:16+08:00,2.66320,51.894623,105.109970
4,2021-05-25 18:46:21+08:00,2.72150,51.894066,105.113968
...,...,...,...,...
249831,2024-10-03 14:37:00+08:00,9.79160,51.847950,104.870730
249832,2024-10-03 14:38:00+08:00,9.79175,51.847950,104.870730
249833,2024-10-03 14:39:00+08:00,9.79290,51.847950,104.870730
249834,2024-10-03 14:40:00+08:00,9.79430,51.847950,104.870730


# Температура поверхности на основе рассматриваемого алгоритма по спутниковым данным

Импорт валидируемой программы. Может потребоваться авторизация в GEE. Смотри коды ошибки при неудаче (там достаточно инструкций).

In [5]:
date_start = df_lst['Time'].min()
date_end = df_lst['Time'].max()
print(date_start, date_end)

2021-05-25 18:42:01+08:00 2024-10-03 14:41:00+08:00


In [11]:
def Cloudmask(image: ee.Image):
   qa = image.select('QA_PIXEL')
   mask = qa.bitwiseAnd(1 << 3).Or(qa.bitwiseAnd(1 << 4))
   return image.updateMask(mask.Not())

def L7_scale_to_celsius(image: ee.image):
   scaled_lst = image.select(['ST_B6']).multiply(0.00341802).subtract(124.15).rename(['LST'])
   return image.addBands(srcImg=scaled_lst)

def L89_scale_to_celsius(image: ee.image):
   scaled_lst = image.select(['ST_B10']).multiply(0.00341802).subtract(124.15).rename(['LST'])
   return image.addBands(srcImg=scaled_lst)

def get_image_list(satellite_name: str):
   match satellite_name:
      case 'Landsat 7 Tier 1':
         coll = ee.ImageCollection("LANDSAT/LE07/C02/T1_L2").map(lambda img: L7_scale_to_celsius(img))
      case 'Landsat 8 Tier 1':
         coll = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2').map(lambda img: L89_scale_to_celsius(img))
      case 'Landsat 9 Tier 1':
         coll = ee.ImageCollection('LANDSAT/LC09/C02/T1_L2').map(lambda img: L89_scale_to_celsius(img))
      case 'Landsat 7 Tier 2':
         coll = ee.ImageCollection("LANDSAT/LE07/C02/T2_L2").map(lambda img: L7_scale_to_celsius(img))
      case 'Landsat 8 Tier 2':
         coll = ee.ImageCollection('LANDSAT/LC08/C02/T2_L2').map(lambda img: L89_scale_to_celsius(img))
      case 'Landsat 9 Tier 2':
         coll = ee.ImageCollection('LANDSAT/LC09/C02/T2_L2').map(lambda img: L89_scale_to_celsius(img))

   coll = coll.filter(
            ee.Filter.date(date_start, date_end)
         ).filterBounds(
            baikal_shape
         ).filter(
            "PROCESSING_LEVEL == 'L2SP'"
         )#.map(Cloudmask)
   if(coll.size().getInfo() == 0):
      return None
   coll = coll.toList(coll.size())
   return coll

In [12]:
def find_entries_for_image(image: ee.Image, df: pd.DataFrame, minutes_padding: int) -> pd.DataFrame:
   timestamp = image.get("system:time_start").getInfo()
   image_dt = pd.to_datetime(timestamp, unit='ms')
   image_dt = image_dt.replace(tzinfo=tz.UTC)

   margin = pd.Timedelta(minutes_padding, 'minutes')

   start_dt = image_dt - margin
   end_dt = image_dt + margin
   print(f'Начало: {start_dt},\nКонец: {end_dt}')
   
   close_entries = df[(start_dt <= df['Time']) & (df['Time'] <= end_dt)]
   close_entries['Image Time'] = image_dt
   return close_entries


def get_validation_entry(
      image: ee.Image, 
      df: pd.DataFrame, 
      minutes_padding: int) -> pd.Series:
   found_entries = find_entries_for_image(image, df, minutes_padding)
   '''Создаёт записи для валидации, рассматривая первое подходящее измерение'''
   if(found_entries.shape[0] == 0): # Если нет строк - не нашли подходящих измерений
      print("Не найдены измерения")
      return None
   print(f"Найдено {found_entries.shape[0]} измерений")
   
   good_entry = None
   for _, entry in found_entries.iterrows():
      longtitude = entry['Lon']
      latitude = entry['Lat']

      region = ee.Geometry.Point([longtitude, latitude])
      image_temperature = image.reduceRegion(
         reducer=ee.Reducer.mean(),
         geometry=region
      ).getInfo()['LST']

      if(image_temperature is not None):
         if(image_temperature > 0):
            # Нашли подходящее измерение
            good_entry = entry
            print(f'Долгота: {longtitude},\nШирота: {latitude}')
            break

   if(good_entry is None):
      return None
   
   good_entry['Image T'] = image_temperature
   return good_entry

def compute_temperatures(
      images: ee.List, 
      df: pd.DataFrame, 
      minutes_padding: int) -> pd.DataFrame:
   result_list = []
   
   size = images.size().getInfo()
   for i in range(0, size):
      print(f"Снимок #{i} из {size}")
      image = ee.Image(images.get(i))
      entry = get_validation_entry(image, df, minutes_padding)
      if(entry is None):
         # это происходит, например когда во время снятия снимка измерений не было
         continue
      elif(entry['Image T'] is None):
         # координата, где было сделано измерение, не попала в снимок (хоть по датам всё сходится)
         print("Координаты измерений лежат вне снимка")
         continue

      print(f"Замеренная температура: {entry['T']}")
      print(f"Предсказанная температура: {entry['Image T']}")
      result_list.append(entry)
      
   return pd.DataFrame(result_list)

def get_validation_data( 
      satellite_name: str, 
      minutes_padding: int):
   Landsat_coll = get_image_list(satellite_name)
   if(Landsat_coll is None):
      # image collection is empty
      return None
   
   computed_df = compute_temperatures(
      Landsat_coll, 
      df_lst, 
      minutes_padding)
   computed_df['Landsat'] = satellite_name

   return computed_df

def get_validation_dataframe(minutes_padding: int, landsats: list[str]) -> pd.DataFrame:
   dataframe_list = []

   for landsat in landsats:
      print("----------")
      print(landsat)
      landsat_df = get_validation_data(landsat, minutes_padding)
      if(landsat_df is None):
         print("Пустая коллекция")
         break
      
      dataframe_list.append(landsat_df)

   return pd.concat(dataframe_list, ignore_index=True)

In [ ]:
result_1 = get_validation_dataframe(1, ['Landsat 7 Tier 1', 'Landsat 8 Tier 1', 'Landsat 9 Tier 1', 'Landsat 7 Tier 2', 'Landsat 8 Tier 2', 'Landsat 9 Tier 2'])
result_1

----------
Landsat 7 Tier 1
Снимок #0 из 472
Начало: 2022-05-18 02:06:09.462000+00:00,
Конец: 2022-05-18 02:08:09.462000+00:00
Не найдены измерения
Снимок #1 из 472
Начало: 2022-06-04 02:05:20.770000+00:00,
Конец: 2022-06-04 02:07:20.770000+00:00
Найдено 2 измерений
Снимок #2 из 472
Начало: 2022-07-08 02:03:13.902000+00:00,
Конец: 2022-07-08 02:05:13.902000+00:00
Не найдены измерения
Снимок #3 из 472


In [9]:
result_15 = get_validation_dataframe(1, ['Landsat 7 Tier 1', 'Landsat 8 Tier 1', 'Landsat 9 Tier 1', 'Landsat 7 Tier 2', 'Landsat 8 Tier 2', 'Landsat 9 Tier 2'])
result_15

----------
Landsat 7 Tier 1
Снимок #0 из 472
Начало: 2022-05-18 02:06:09.462000+00:00,
Конец: 2022-05-18 02:08:09.462000+00:00
Не найдены измерения
Снимок #1 из 472
Начало: 2022-06-04 02:05:20.770000+00:00,
Конец: 2022-06-04 02:07:20.770000+00:00
Найдено 2 измерений
Снимок #2 из 472
Начало: 2022-07-08 02:03:13.902000+00:00,
Конец: 2022-07-08 02:05:13.902000+00:00
Не найдены измерения
Снимок #3 из 472
Начало: 2022-07-25 02:01:56.870000+00:00,
Конец: 2022-07-25 02:03:56.870000+00:00
Не найдены измерения
Снимок #4 из 472
Начало: 2022-10-18 01:53:24.328000+00:00,
Конец: 2022-10-18 01:55:24.328000+00:00
Не найдены измерения
Снимок #5 из 472
Начало: 2023-04-21 01:31:54.832000+00:00,
Конец: 2023-04-21 01:33:54.832000+00:00
Не найдены измерения
Снимок #6 из 472
Начало: 2023-05-13 01:29:41.391000+00:00,
Конец: 2023-05-13 01:31:41.391000+00:00
Не найдены измерения
Снимок #7 из 472
Начало: 2023-06-04 01:26:51.703000+00:00,
Конец: 2023-06-04 01:28:51.703000+00:00
Не найдены измерения
Снимок #8 из 

,Time,T,Lat,Lon,Image Time,Image T,Landsat
0,2021-09-23 10:35:16+08:00,9.57190,53.143623,107.849472,2021-09-23 02:36:07.773000+00:00,8.978461,Landsat 7 Tier 1
1,2022-08-04 10:13:45+08:00,14.13260,55.648727,109.369957,2022-08-04 02:13:44.191000+00:00,18.029378,Landsat 7 Tier 1
2,2021-07-12 10:47:50+08:00,16.13070,53.470188,108.915337,2021-07-12 02:48:42.341000+00:00,1.181957,Landsat 7 Tier 1
3,2021-07-28 10:47:17+08:00,8.69340,53.017338,107.355103,2021-07-28 02:47:19.296000+00:00,8.366635,Landsat 7 Tier 1
4,2021-08-13 10:45:21+08:00,13.08410,53.156502,107.977829,2021-08-13 02:45:54.146000+00:00,11.661607,Landsat 7 Tier 1
5,2021-08-20 10:51:51+08:00,14.13620,51.610619,104.894226,2021-08-20 02:51:51.804000+00:00,13.910664,Landsat 7 Tier 1
6,2021-07-26 10:59:37+08:00,14.77180,51.628082,104.103561,2021-07-26 03:00:15.278000+00:00,13.415051,Landsat 7 Tier 1
7,2021-07-13 11:38:04+08:00,7.07820,55.332798,109.608627,2021-07-13 03:38:29.542000+00:00,7.505294,Landsat 8 Tier 1
8,2021-07-13 11:38:04+08:00,7.07820,55.332798,109.608627,2021-07-13 03:38:53.420000+00:00,7.512130,Landsat 8 Tier 1
9,2021-07-29 11:39:00+08:00,11.98860,53.443645,108.509331,2021-07-29 03:39:24.750000+00:00,12.772463,Landsat 8 Tier 1


In [ ]:
result_1.to_csv('Данные_валидации/raw/ls2p_validation1_no_mask.csv', index=False)
# result_15.to_csv('Данные_валидации/raw/ls2p_validation15.csv', index=False)
result_1.to_csv('Данные_валидации/ready/ls2p_validation1_no_mask.csv', index=False)
# result_15.to_csv('Данные_валидации/ready/ls2p_validation15.csv', index=False)